# Pattern Mining Phase: Matrix Profile Analysis

To validate the approximate motifs detected by SAX, a second high-precision pattern mining stage was conducted using the **Matrix Profile** method, implemented via the `stumpy` Python library. Unlike SAX, which relies on discretization, the Matrix Profile computes the exact Euclidean distances between all subsequences, offering a parameter-free method to identify motifs (repeating patterns) and discords (anomalies).

### Methodology
The Matrix Profile was computed for the Earthquake Time Series (E, N, and V channels) to locate the most conserved waveform shapes.

* **Algorithm:** We utilized `stumpy.stump`, a highly parallelized implementation of the exact motif discovery algorithm.
* **Window Size ($m$):** A window size of $m=1000,$ (approx. 10 second) was selected, consistent with the stable motif duration identified in the SAX analysis.
* **Metric:** **Z-normalized Euclidean Distance**. This normalization is critical for seismic analysis as it focuses on waveform *shape* rather than absolute *amplitude*, allowing the detection of repeating scattering patterns even as the earthquake signal attenuates over time.



### Analytical Objectives
The Matrix Profile vector $P$ was analyzed to extract two key physical features:

1.  **Motif Discovery (Global Minima):**
    The indices of the minimum values in $P$ correspond to the **Top-1 Motif**—the pair of subsequences with the highest similarity. This identifies the "signature" waveform of the site's crustal response.

2.  **Regime Change Detection (Semantic Segmentation):**
    By analyzing transitions in the Matrix Profile values, we identified boundaries between different physical regimes (e.g., the transition from the chaotic P-onset to the rhythmic, repeating Coda phase).

### Comparison with SAX Results
This stage serves as a verification step. While SAX provides a global statistical view of symbol distribution, `stumpy` provides exact localization.
* **Expectation:** If the SAX "P-coda motif" is real, the Matrix Profile should show a distinct "valley" (low distance values) in the region between P and S arrivals, indicating high self-similarity.
* **Discords:** High values in the Matrix Profile will highlight non-repeating transients, expected to correspond to the unique impulsive onsets of the P and S phases.



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

signal_data = np.load("signal_data/processed_seismic_data.npy")

print(signal_data.shape)

(29871, 3, 1000)


In [3]:
metadata = pd.read_csv("metadata/processed_metadata.csv")

print(metadata.shape)

metadata.head()

(29871, 28)


,trace_name,network_code,receiver_code,receiver_type,source_origin_time,trace_start_time,receiver_latitude,receiver_longitude,receiver_elevation_m,p_arrival_sample,...,source_magnitude,source_magnitude_type,source_distance_km,back_azimuth_deg,coda_end_sample,id,norm_s_arrival_sample,snr_db_E,snr_db_N,snr_db_V
0,109C.TA_20061103155652_EV,TA,109C,BH,2006-11-03 15:56:42.73,2006-11-03 15:56:53.610000,32.8889,-117.1051,150.0,600.0,...,4.3,mb,101.34,281.7,5508,235427,236,65.000000,65.500000,61.400002
1,109C.TA_20061129211102_EV,TA,109C,BH,2006-11-29 21:10:55.02,2006-11-29 21:11:03.890000,32.8889,-117.1051,150.0,900.0,...,4.1,ml,108.03,273.8,3199,235432,558,55.000000,56.099998,43.200001
2,109C.TA_20061129221547_EV,TA,109C,BH,2006-11-29 22:15:38.65,2006-11-29 22:15:48.630000,32.8889,-117.1051,150.0,800.0,...,3.9,ml,106.69,273.7,5252,235434,283,49.000000,48.000000,39.200001
3,109C.TA_20070209033349_EV,TA,109C,BH,2007-02-09 03:33:42.80,2007-02-09 03:33:50.600000,32.8889,-117.1051,150.0,900.0,...,4.2,ml,98.93,246.8,2866,235437,580,65.000000,68.199997,58.700001
4,109C.TA_20070415225732_EV,TA,109C,BH,2007-04-15 22:57:25.78,2007-04-15 22:57:33.940000,32.8889,-117.1051,150.0,900.0,...,4.3,ml,99.46,280.3,5848,235441,240,60.099998,64.800003,53.400002


In [4]:
print(metadata.columns)

Index(['trace_name', 'network_code', 'receiver_code', 'receiver_type',
       'source_origin_time', 'trace_start_time', 'receiver_latitude',
       'receiver_longitude', 'receiver_elevation_m', 'p_arrival_sample',
       'p_status', 'p_travel_sec', 's_arrival_sample', 's_status', 'source_id',
       'source_latitude', 'source_longitude', 'source_depth_km',
       'source_magnitude', 'source_magnitude_type', 'source_distance_km',
       'back_azimuth_deg', 'coda_end_sample', 'id', 'norm_s_arrival_sample',
       'snr_db_E', 'snr_db_N', 'snr_db_V'],
      dtype='object')


In [5]:
import stumpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tqdm import tqdm
import os

# ================================================================
# 0. Ensure output directory exists
# ================================================================
os.makedirs("motif_results", exist_ok=True)

# ================================================================
# 1. PARAMETERS
# ================================================================
MOTIF_LENGTHS = [100, 200, 500, 1000]
K_RANGE = range(2, 10)  # Elbow curve range
N_CLUSTERS = 4          # Default cluster count
N_EXAMPLES = 10         # Motifs to plot per cluster

# signal_data shape must be (N, 3, T)
signal_data = signal_data.astype(np.float64)
N, C, T = signal_data.shape


# ================================================================
# 2. LOOP OVER MOTIF LENGTHS
# ================================================================
for m in MOTIF_LENGTHS:
    print(f"\n===============================")
    print(f"   PROCESSING MOTIF LENGTH m={m}")
    print(f"===============================\n")

    save_dir = f"motif_results/m_{m}"
    os.makedirs(save_dir, exist_ok=True)

    # ================================================================
    # 2A. Extract motifs for all signals using mstump
    # ================================================================
    motif_shapes = []      # will be (N, 3, m)
    motif_dists = []

    for i in tqdm(range(N), desc=f"Motifs m={m}"):
        signal = signal_data[i]

        # compute multidimensional matrix profile
        mp, _ = stumpy.mstump(signal, m)
        
        motif_idx = np.argmin(mp[0])
        start, end = motif_idx, motif_idx + m
        
        motif_segment = signal[:, start:end]   # (3, m)
        motif_shapes.append(motif_segment)
        motif_dists.append(mp[0, motif_idx])

    motif_shapes = np.array(motif_shapes)
    motifs_flat = motif_shapes.reshape(N, C * m)

    # Save motifs in case needed later
    np.save(f"{save_dir}/motifs.npy", motif_shapes)
    np.save(f"{save_dir}/motif_distances.npy", motif_dists)


    # ================================================================
    # 2B. K-Means ELBOW CURVE
    # ================================================================
    sse = []
    for k in tqdm(K_RANGE, desc=f"Elbow m={m}"):
        km = KMeans(n_clusters=k, random_state=42)
        km.fit(motifs_flat)
        sse.append(km.inertia_)

    plt.figure(figsize=(7, 5))
    plt.plot(K_RANGE, sse, marker="o")
    plt.xlabel("Number of Clusters (k)")
    plt.ylabel("SSE (Inertia)")
    plt.title(f"Elbow Curve for m={m}")
    plt.grid()
    plt.tight_layout()
    plt.savefig(f"{save_dir}/elbow_m{m}.png")
    plt.close()


    # ================================================================
    # 2C. Cluster with N_CLUSTERS and plot centroids
    # ================================================================
    kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42)
    labels = kmeans.fit_predict(motifs_flat)

    np.save(f"{save_dir}/labels.npy", labels)

    fig, axes = plt.subplots(C, N_CLUSTERS, figsize=(5*N_CLUSTERS, 10))

    if C == 1:  # fix plot layout for single channel case
        axes = np.expand_dims(axes, axis=0)

    for cluster_id in range(N_CLUSTERS):
        centroid = kmeans.cluster_centers_[cluster_id].reshape(C, m)
        cluster_idxs = np.where(labels == cluster_id)[0]

        for ch in range(C):
            ax = axes[ch, cluster_id]
            ax.plot(centroid[ch], linewidth=3, label="Centroid")

            # plot examples
            sample_idxs = (
                np.random.choice(cluster_idxs, N_EXAMPLES, replace=False)
                if len(cluster_idxs) > N_EXAMPLES else cluster_idxs
            )

            for idx in sample_idxs:
                ax.plot(motif_shapes[idx][ch], alpha=0.4)

            ax.set_title(f"Cluster {cluster_id} — Channel {ch}")
            ax.set_xlabel("Time")
            ax.set_ylabel("Amp")
            ax.legend()

    plt.tight_layout()
    plt.savefig(f"{save_dir}/clusters_m{m}.png")
    plt.close()

    print(f"Saved: {save_dir}/clusters_m{m}.png")
    print(f"Saved: {save_dir}/elbow_m{m}.png")
    print(f"Motif extraction for m={m} complete.\n")


print("\nALL MOTIF LENGTHS FINISHED SUCCESSFULLY ✔")



   PROCESSING MOTIF LENGTH m=100



Elbow m=100: 100%|██████████| 8/8 [00:06<00:00,  1.24it/s]


Saved: motif_results/m_100/clusters_m100.png
Saved: motif_results/m_100/elbow_m100.png
Motif extraction for m=100 complete.


   PROCESSING MOTIF LENGTH m=200



Elbow m=200: 100%|██████████| 8/8 [00:06<00:00,  1.24it/s]


Saved: motif_results/m_200/clusters_m200.png
Saved: motif_results/m_200/elbow_m200.png
Motif extraction for m=200 complete.


   PROCESSING MOTIF LENGTH m=500



Elbow m=500: 100%|██████████| 8/8 [00:23<00:00,  2.98s/it]


Saved: motif_results/m_500/clusters_m500.png
Saved: motif_results/m_500/elbow_m500.png
Motif extraction for m=500 complete.


   PROCESSING MOTIF LENGTH m=1000



Elbow m=1000: 100%|██████████| 8/8 [01:29<00:00, 11.17s/it]


Saved: motif_results/m_1000/clusters_m1000.png
Saved: motif_results/m_1000/elbow_m1000.png
Motif extraction for m=1000 complete.


ALL MOTIF LENGTHS FINISHED SUCCESSFULLY ✔


In [6]:
import stumpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from tqdm import tqdm
import os

# ================================================================
# 0. Ensure output directory exists
# ================================================================
os.makedirs("motif_results_adv", exist_ok=True)

# ================================================================
# 1. PARAMETERS
# ================================================================
MOTIF_LENGTHS = [100, 200, 500, 1000]
K_RANGE = range(2, 15)   # Auto K search range
N_EXAMPLES = 10          # Motifs to plot per cluster

signal_data = signal_data.astype(np.float64)
N, C, T = signal_data.shape

# ================================================================
# 2. LOOP OVER MOTIF LENGTHS
# ================================================================
for m in MOTIF_LENGTHS:
    print(f"\n===============================")
    print(f"   PROCESSING MOTIF LENGTH m={m}")
    print(f"===============================\n")

    save_dir = f"motif_results/m_{m}"
    os.makedirs(save_dir, exist_ok=True)

    # ================================================================
    # 2A. EXTRACT MOTIFS USING MSTUMP
    # ================================================================
    motif_shapes = []
    motif_dists = []

    for i in tqdm(range(N), desc=f"Motifs m={m}"):
        signal = signal_data[i]
        mp, _ = stumpy.mstump(signal, m)

        motif_idx = np.argmin(mp[0])
        start, end = motif_idx, motif_idx + m

        motif_segment = signal[:, start:end]
        motif_shapes.append(motif_segment)
        motif_dists.append(mp[0, motif_idx])

    motif_shapes = np.array(motif_shapes)
    motifs_flat = motif_shapes.reshape(N, C * m)

    np.save(f"{save_dir}/motifs.npy", motif_shapes)
    np.save(f"{save_dir}/motif_distances.npy", motif_dists)

    # ================================================================
    # 2B. AUTO-BEST-K SELECTION
    # ================================================================
    sse, sil_scores, ch_scores = [], [], []

    for k in tqdm(K_RANGE, desc=f"Auto-K m={m}"):
        km = KMeans(n_clusters=k, random_state=42)
        labels = km.fit_predict(motifs_flat)
        sse.append(km.inertia_)

        if len(set(labels)) > 1:
            sil_scores.append(silhouette_score(motifs_flat, labels))
            ch_scores.append(calinski_harabasz_score(motifs_flat, labels))
        else:
            sil_scores.append(-1)
            ch_scores.append(-1)

    # SELECT BEST K
    best_k_sil = K_RANGE[np.argmax(sil_scores)]
    best_k_ch = K_RANGE[np.argmax(ch_scores)]

    # Combine votes → most robust
    best_k = int(np.median([best_k_sil, best_k_ch]))
    print(f"\nAUTO SELECTED BEST k = {best_k}")

    # Save elbow, silhouette, CH
    plt.figure(figsize=(7, 5))
    plt.plot(K_RANGE, sse, marker="o")
    plt.title(f"Elbow Curve (m={m})")
    plt.xlabel("k"); plt.ylabel("SSE")
    plt.grid(); plt.tight_layout()
    plt.savefig(f"{save_dir}/elbow_m{m}.png")
    plt.close()

    plt.figure(figsize=(7, 5))
    plt.plot(K_RANGE, sil_scores, marker="o")
    plt.title(f"Silhouette Scores (m={m})")
    plt.xlabel("k"); plt.ylabel("Score")
    plt.grid(); plt.tight_layout()
    plt.savefig(f"{save_dir}/silhouette_m{m}.png")
    plt.close()

    plt.figure(figsize=(7, 5))
    plt.plot(K_RANGE, ch_scores, marker="o")
    plt.title(f"Calinski-Harabasz Scores (m={m})")
    plt.xlabel("k"); plt.ylabel("Score")
    plt.grid(); plt.tight_layout()
    plt.savefig(f"{save_dir}/ch_m{m}.png")
    plt.close()

    # ================================================================
    # 2C. FINAL CLUSTERING WITH BEST K
    # ================================================================
    kmeans = KMeans(n_clusters=best_k, random_state=42)
    labels = kmeans.fit_predict(motifs_flat)

    np.save(f"{save_dir}/labels_kmeans.npy", labels)

    # ================================================================
    # 2D. PCA 2D + 3D
    # ================================================================
    pca2 = PCA(n_components=2).fit_transform(motifs_flat)
    pca3 = PCA(n_components=3).fit_transform(motifs_flat)

    # 2D PCA
    plt.figure(figsize=(8, 6))
    for k in range(best_k):
        idxs = np.where(labels == k)[0]
        plt.scatter(pca2[idxs, 0], pca2[idxs, 1], label=f"C{k}", alpha=0.6)
    plt.legend(); plt.title(f"PCA 2D (m={m})")
    plt.tight_layout()
    plt.savefig(f"{save_dir}/pca2_m{m}.png")
    plt.close()

    # 3D PCA
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    for k in range(best_k):
        idxs = np.where(labels == k)[0]
        ax.scatter(pca3[idxs, 0], pca3[idxs, 1], pca3[idxs, 2], label=f"C{k}", alpha=0.5)
    ax.set_title(f"PCA 3D (m={m})")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{save_dir}/pca3_m{m}.png")
    plt.close()

    # ================================================================
    # 2E. HIERARCHICAL CLUSTERING (WARD)
    # ================================================================
    hier = AgglomerativeClustering(n_clusters=best_k, linkage="ward")
    labels_hier = hier.fit_predict(motifs_flat)
    np.save(f"{save_dir}/labels_hier.npy", labels_hier)

    # PCA 2D Visual Hierarchical
    plt.figure(figsize=(8, 6))
    for k in range(best_k):
        idxs = np.where(labels_hier == k)[0]
        plt.scatter(pca2[idxs, 0], pca2[idxs, 1], label=f"H{k}", alpha=0.6)
    plt.legend(); plt.title(f"Hierarchical PCA 2D (m={m})")
    plt.tight_layout()
    plt.savefig(f"{save_dir}/pca2_hier_m{m}.png")
    plt.close()

    # ================================================================
    # 2F. PLOT CLUSTER CENTROIDS + EXAMPLES + COUNT
    # ================================================================
    fig, axes = plt.subplots(C, best_k, figsize=(6*best_k, 10))

    if C == 1:
        axes = np.expand_dims(axes, 0)

    for cluster_id in range(best_k):
        centroid = kmeans.cluster_centers_[cluster_id].reshape(C, m)
        cluster_idxs = np.where(labels == cluster_id)[0]
        count = len(cluster_idxs)

        for ch in range(C):
            ax = axes[ch, cluster_id]

            ax.plot(centroid[ch], linewidth=3, label="Centroid", color="black")

            sample_idxs = (
                np.random.choice(cluster_idxs, N_EXAMPLES, replace=False)
                if count > N_EXAMPLES else cluster_idxs
            )

            for idx in sample_idxs:
                ax.plot(motif_shapes[idx][ch], alpha=0.4)

            ax.set_title(f"Cluster {cluster_id} (n={count}) — Ch {ch}")
            ax.grid()
            ax.legend()

    plt.tight_layout()
    plt.savefig(f"{save_dir}/clusters_m{m}.png")
    plt.close()

    print(f"✔ Saved centroid plots m={m}")
    print(f"✔ Auto-K = {best_k} finished\n")

print("\n===============================")
print(" ALL MOTIF LENGTHS FINISHED ✔ ")
print("===============================")



   PROCESSING MOTIF LENGTH m=100



Auto-K m=100: 100%|██████████| 13/13 [03:22<00:00, 15.61s/it]



AUTO SELECTED BEST k = 3
✔ Saved centroid plots m=100
✔ Auto-K = 3 finished


   PROCESSING MOTIF LENGTH m=200



Auto-K m=200: 100%|██████████| 13/13 [04:07<00:00, 19.05s/it]



AUTO SELECTED BEST k = 3
✔ Saved centroid plots m=200
✔ Auto-K = 3 finished


   PROCESSING MOTIF LENGTH m=500



Auto-K m=500: 100%|██████████| 13/13 [07:02<00:00, 32.48s/it]



AUTO SELECTED BEST k = 2
✔ Saved centroid plots m=500
✔ Auto-K = 2 finished


   PROCESSING MOTIF LENGTH m=1000



Auto-K m=1000: 100%|██████████| 13/13 [12:36<00:00, 58.16s/it]



AUTO SELECTED BEST k = 8
✔ Saved centroid plots m=1000
✔ Auto-K = 8 finished


 ALL MOTIF LENGTHS FINISHED ✔ 


# Code Explanation

We applied multidimensional motif discovery and clustering to a dataset of **three-channel seismic signals** using motif lengths of 100, 200, 500, and 1000 time steps. The results reported here focus on the **1000-length motifs**, which produced the most stable and interpretable cluster structures.

## 1. Motif Extraction

Using **multidimensional matrix profiles (mstump)**, a representative motif was extracted from every signal across all three channels. These motifs served as compressed, shape-based descriptors of the dominant local pattern within each waveform.

## 2. Automatic Cluster Selection

For each motif length, we performed a systematic evaluation of candidate cluster numbers using:

* **Within-cluster SSE (Elbow)**
* **Silhouette Score**
* **Calinski–Harabasz Index**

The optimal number of clusters was determined by combining these metrics via a median-vote heuristic.
For the 1000-step motifs, the best value converged to **k = 8 clusters**, indicating that the dataset contains approximately eight distinct characteristic waveform shapes.

## 3. Cluster Structure and Centroid Patterns

The cluster visualizations show each cluster's centroid (black curve) along with a large number of motif samples overlaid with transparency. Several strong trends emerged:

### (a) Cluster Size Variation

Clusters vary significantly in population:

* Largest clusters contain **5000–6000 signals**
* Smallest clusters contain **2500–3000 signals**

This indicates that certain motif shapes occur far more frequently in the dataset.

### (b) High Within-Cluster Agreement

Many clusters display extremely tight overlay around the centroid, especially in:

* **Channel 1** and **Channel 2**
* The **mid-segment region** (≈ 300–800)

This consistency suggests that the motifs capture a robust and repeated structural pattern in the underlying signals.

### (c) Presence of Noise-Dominated Clusters

A few clusters exhibit:

* Larger variance envelopes
* Weakly defined centroid features
* High variability in amplitude

These clusters likely correspond to **noisy signals**, **non-standard events**, or motifs that do not contain the canonical structure seen elsewhere.

### (d) Multi-Channel Synchrony

Across all clusters, the three channels tend to share:

* Similar peak timings
* Similar decay phases
* Aligned transient features

This validates that the motifs are capturing a **coherent multivariate event** rather than unrelated channel-specific fluctuations.

### (e) Distinctive Onset Characteristics

The main differences between clusters arise from:

* Amplitude and sign of the initial transient (first ~150 samples)
* Strength of the peak around the motif center (~400–600 samples)
* Noise level and decay slope in the final third

These distinctions allow the clustering to separate signals not only by shape but also by subtle temporal dynamics.

## 4. PCA Visualization

PCA 2D and 3D projections show clearly separable cluster groups.
Most clusters form well-bounded ellipsoidal clouds, supporting that:

* The centroid shapes represent **distinct waveform families**

Some overlaps occur between noise-heavy clusters, consistent with the less defined centroid structures observed.

## 5. Hierarchical Clustering

Hierarchical (Ward) clustering produces a similar structure to KMeans, but with slightly smoother boundaries.
Subclusters inside each major group indicate that motifs may have additional **micro-patterns** suitable for sub-clustering.

